In [1]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import math
import copy as cp

In [47]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import math
import copy as cp
df1=pd.read_csv("first_round_training_data.csv")
df2=pd.read_csv("first_round_testing_data.csv")
train_x=np.zeros([6000,10],dtype=np.float32)
train_y=np.zeros([6000],dtype=np.int32)

test_id=np.zeros([6000],dtype=np.int32)
test_x=np.zeros([6000,10],dtype=np.float32)
cls2int={"Excellent":0,"Good":1,"Pass":2,"Fail":3}
for i in range(1,11):
    par="Parameter"+str(i)
    tmp=df1[par]
    for j in range(6000):
        train_x[j,i-1]=tmp[j]
        cls=cls2int[df1["Quality_label"][j]]
        train_y[j]=cls
for i in range(1,11):
    par="Parameter"+str(i)
    tmp=df2[par]
    for j in range(6000):
        test_x[j,i-1]=tmp[j]
        ID=int(df2["Group"][j])
        test_id[j]=ID
d_train = pd.read_csv("first_round_training_data.csv")
d_test = pd.read_csv("first_round_testing_data.csv")

########################################################################################################################
# ORI: 处理类别标签并分割x,y
#
# for i in range(1, 11):
#     par = "Parameter" + str(i)
#     tmp = d_train[par]
#     for j in range(6000):
#         x_train[j, i - 1] = tmp[j]
#         cls = cls2int[d_train["Quality_label"][j]]
#         y_train[j] = cls
# lgb_train = lgb.Dataset(x_train, y_train)

lgb_train = lgb.Dataset(
    d_train[["Parameter" + str(i) for i in range(1, 11)]],
    d_train["Quality_label"].map({"Excellent": 0, "Good": 1, "Pass": 2, "Fail": 3}),
)
########################################################################################################################


x_test = d_test.drop(["Group"], axis=1)
########################################################################################################################

# lgbm params
params = {
'boosting_type': 'gbdt',
'objective': 'multiclassova',
'num_class': 4,
'metric': 'multi_error',
'num_leaves': 63,
'learning_rate': 0.01,
'feature_fraction': 0.9,
'bagging_fraction': 0.9,
'bagging_seed':0,
'bagging_freq': 1,
'verbose': -1,
'reg_alpha':1,
'reg_lambda':2,
'lambda_l1': 0,
'lambda_l2': 1,
'num_threads': 8,
    "boosting_type": "gbdt",
    "objective": "multiclassova",
    "num_class": 4,
    "metric": "multi_error",
    "num_leaves": 63,
    "learning_rate": 0.01,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.9,
    "bagging_seed": 0,
    "bagging_freq": 1,
    "verbose": -1,
    "reg_alpha": 1,
    "reg_lambda": 2,
    "lambda_l1": 0,
    "lambda_l2": 1,
    "num_threads": 8,
}
lgb_train = lgb.Dataset(train_x, train_y)
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=1300,
                valid_sets=[lgb_train],
                valid_names=['train'],
                verbose_eval=100,
                )
f=open("lgb1300round.csv","w")
ans=gbm.predict(test_x, num_iteration=1300)
tmp=np.zeros([120,4])
cnt=np.zeros([120])
for i in range(6000):
    ID=test_id[i]
    tmp[ID,:]+=ans[i,:]
    cnt[ID]+=1
for i in range(120):
    SUM=np.sum(tmp[i,:])
    tmp[i,:]/=SUM

f.write("Group,Excellent ratio,Good ratio,Pass ratio,Fail ratio\n")
for i in range(120):
    f.write(str(i))
    for j in range(4):
        f.write(","+str(tmp[i,j]))
    f.write("\n")
f.close()
# train
gbm = lgb.train(
    params,
    lgb_train,
    num_boost_round=1300,
    valid_sets=[lgb_train],
    valid_names=["train"],
    verbose_eval=100,
)

# predict
y_pred = gbm.predict(d_train[["Parameter" + str(i) for i in range(1, 11)]], num_iteration=1300)



[100]	train's multi_error: 0.363
[200]	train's multi_error: 0.296333
[300]	train's multi_error: 0.251667
[400]	train's multi_error: 0.208
[500]	train's multi_error: 0.1705
[600]	train's multi_error: 0.138333
[700]	train's multi_error: 0.109167
[800]	train's multi_error: 0.0861667
[900]	train's multi_error: 0.0676667
[1000]	train's multi_error: 0.0528333
[1100]	train's multi_error: 0.0395
[1200]	train's multi_error: 0.0305
[1300]	train's multi_error: 0.0236667
[100]	train's multi_error: 0.363
[200]	train's multi_error: 0.296333
[300]	train's multi_error: 0.251667
[400]	train's multi_error: 0.208
[500]	train's multi_error: 0.1705
[600]	train's multi_error: 0.138333
[700]	train's multi_error: 0.109167
[800]	train's multi_error: 0.0861667
[900]	train's multi_error: 0.0676667
[1000]	train's multi_error: 0.0528333
[1100]	train's multi_error: 0.0395
[1200]	train's multi_error: 0.0305
[1300]	train's multi_error: 0.0236667


In [46]:
train_y_ =  train_y[4000:]
trueCount = 0
for i, c in enumerate(y_pred):
    c = list(c)
    if c.index(max(c)) == train_y_[i]:
        trueCount += 1
print(trueCount/(len(train_y_)))

0.49


(2000,)

In [48]:
########################################################################################################################

y_pred = pd.DataFrame(
    y_pred, columns=["Excellent ratio", "Good ratio", "Pass ratio", "Fail ratio"]
)

y_pred["Group"] = d_test["Group"]
y_pred

,Excellent ratio,Good ratio,Pass ratio,Fail ratio,Group
0,0.039827,0.097096,0.693268,0.108702,0
1,0.034100,0.048366,0.077778,0.912901,0
2,0.025041,0.191337,0.047594,0.875257,0
3,0.013964,0.075628,0.060510,0.937155,0
4,0.017919,0.056532,0.137953,0.893410,0
5,0.047126,0.432880,0.134948,0.166922,0
6,0.022061,0.731592,0.093911,0.067303,0
7,0.082646,0.662978,0.082663,0.029447,0
8,0.034784,0.190394,0.581345,0.040547,0
9,0.068215,0.789466,0.092436,0.037983,0


In [50]:
group_pred = y_pred.groupby("Group").mean()
for c in ["Excellent ratio", "Good ratio", "Pass ratio", "Fail ratio"]:
    group_pred[c] /= group_pred.sum(axis=1)
########################################################################################################################
group_pred.to_csv("lgb_submit.csv")

,Excellent ratio,Good ratio,Pass ratio,Fail ratio
Group,,,,
0,0.320945,0.246462,0.293107,0.128842
1,0.195081,0.217709,0.490045,0.109634
2,0.096985,0.099819,0.696582,0.112655
3,0.263487,0.223039,0.246209,0.286693
4,0.228989,0.269821,0.331099,0.144514
5,0.098671,0.155155,0.740989,0.021560
6,0.136724,0.261681,0.366814,0.232867
7,0.147703,0.206357,0.542259,0.066851
8,0.114788,0.150213,0.648051,0.080812


In [59]:
group_pred

,Excellent ratio,Good ratio,Pass ratio,Fail ratio
Group,,,,
0,0.320945,0.246462,0.293107,0.128842
1,0.195081,0.217709,0.490045,0.109634
2,0.096985,0.099819,0.696582,0.112655
3,0.263487,0.223039,0.246209,0.286693
4,0.228989,0.269821,0.331099,0.144514
5,0.098671,0.155155,0.740989,0.021560
6,0.136724,0.261681,0.366814,0.232867
7,0.147703,0.206357,0.542259,0.066851
8,0.114788,0.150213,0.648051,0.080812
